# Vehicle Damage Detection - Basic Usage Guide

This notebook demonstrates basic usage of the Vehicle Damage Detection System.

## Overview

The system can:
- Detect vehicle damage using YOLO models
- Classify damage severity (minor/moderate/severe/critical)
- Estimate repair costs
- Provide visualization of detected damage

## Prerequisites

Make sure you have the system running:
```bash
# Using Docker Compose (recommended)
docker-compose up -d

# Or run locally
# Terminal 1: uvicorn src.api.main:app --reload
# Terminal 2: streamlit run src/ui/app.py
```

In [ ]:
# Import required libraries
import requests
import json
from PIL import Image
import io
import matplotlib.pyplot as plt
import base64

# Configuration
API_URL = "http://localhost:8000"
UI_URL = "http://localhost:8501"

print(f"API URL: {API_URL}")
print(f"UI URL: {UI_URL}")

## 1. Health Check

In [ ]:
# Check system health
response = requests.get(f"{API_URL}/health")
print(f"Status Code: {response.status_code}")

if response.status_code == 200:
    health_data = response.json()
    print("\nSystem Health:")
    for key, value in health_data.items():
        print(f"  {key}: {value}")
else:
    print("❌ System is not healthy")

## 2. Analyze an Image

In [ ]:
# Function to analyze an image
def analyze_image(image_path):
    """Analyze an image for vehicle damage."""
    
    with open(image_path, "rb") as image_file:
        files = {"file": (image_path, image_file, "image/jpeg")}
        
        print(f"Analyzing image: {image_path}")
        response = requests.post(f"{API_URL}/api/analyze", files=files)
        
        if response.status_code == 200:
            result = response.json()
            print("✅ Analysis completed successfully!")
            return result
        else:
            print(f"❌ Analysis failed: {response.status_code}")
            print(f"Error: {response.text}")
            return None

# Example usage - replace with your image path
# result = analyze_image("path/to/your/car/image.jpg")

## 3. Display Analysis Results

In [ ]:
def display_analysis_result(result):
    """Display analysis results in a formatted way."""
    
    if not result:
        print("No result to display")
        return
    
    # Extract key information
    classification = result.get("classification", {})
    cost_estimate = result.get("cost_estimate", {})
    detection = result.get("detection", {})
    
    print("=" * 50)
    print("🚗 VEHICLE DAMAGE ANALYSIS RESULTS")
    print("=" * 50)
    
    # Damage Detection
    print(f"\n🔍 DETECTION RESULTS:")
    print(f"   • Damages detected: {classification.get('damage_count', 0)}")
    print(f"   • Detection confidence: {classification.get('avg_confidence', 0)*100:.1f}%")
    print(f"   • Total processing time: {result.get('total_processing_time', 0):.2f}s")
    
    # Damage Classification
    severity = classification.get('severity')
    if severity:
        severity_emoji = {
            'minor': '🟢',
            'moderate': '🟡', 
            'severe': '🟠',
            'critical': '🔴'
        }.get(severity, '❓')
        
        print(f"\n⚠️  SEVERITY ASSESSMENT:")
        print(f"   • Level: {severity_emoji} {severity.upper()}")
        print(f"   • Damage coverage: {classification.get('area_ratio', 0)*100:.1f}% of image")
        
        # Damage types
        damage_types = classification.get('damage_types', {})
        if damage_types:
            print(f"\n🔧 DAMAGE TYPES:")
            for dmg_type, count in damage_types.items():
                print(f"   • {dmg_type}: {count}")
    
    # Cost Estimation
    estimated_cost = cost_estimate.get('estimated_cost', 0)
    if estimated_cost > 0:
        print(f"\n💰 COST ESTIMATION:")
        print(f"   • Estimated cost: ${estimated_cost:,.2f} {cost_estimate.get('currency', 'USD')}")
        print(f"   • Cost range: ${cost_estimate.get('min_cost', 0):,.2f} - ${cost_estimate.get('max_cost', 0):,.2f}")
        print(f"   • Parts cost: ${cost_estimate.get('parts_cost', 0):,.2f}")
        print(f"   • Labor cost: ${cost_estimate.get('labor_cost', 0):,.2f}")
    else:
        print(f"\n💰 COST ESTIMATION:")
        print("   • No damage detected - no repair costs")
    
    print("\n" + "=" * 50)

# Example usage
# if result:
#     display_analysis_result(result)

## 4. Get Analysis History

In [ ]:
# Get analysis history
def get_analysis_history(limit=10):
    """Get the history of analysis results."""
    
    response = requests.get(f"{API_URL}/api/history?limit={limit}")
    
    if response.status_code == 200:
        history = response.json()
        print(f"📊 ANALYSIS HISTORY (Last {len(history)} results):")
        print("-" * 60)
        
        for item in history:
            created_at = item.get('created_at', '')[:19]  # Remove timezone
            severity = item.get('severity', 'N/A')
            cost = item.get('estimated_cost', 0)
            currency = item.get('currency', 'USD')
            
            print(f"ID: {item.get('id'):>3} | {created_at} | {severity:>8} | ${cost:>8.0f} {currency}")
        
        return history
    else:
        print(f"❌ Failed to get history: {response.status_code}")
        return []

# Get history
# history = get_analysis_history(5)

## 5. Complete Example Workflow

In [ ]:
def complete_workflow(image_path):
    """Complete workflow: check health, analyze, and display results."""
    
    print("🚀 VEHICLE DAMAGE ANALYSIS WORKFLOW")
    print("\n1. Checking system health...")
    
    # Health check
    health_response = requests.get(f"{API_URL}/health")
    if health_response.status_code != 200:
        print("❌ System is not healthy. Please check if the API is running.")
        return
    
    health = health_response.json()
    print(f"   ✅ System: {health.get('status', 'unknown')}")
    print(f"   ✅ Model: {'Loaded' if health.get('model_loaded') else 'Not loaded'}")
    print(f"   ✅ Database: {'Connected' if health.get('database_connected') else 'Not connected'}")
    
    print("\n2. Analyzing image...")
    
    # Analyze image
    result = analyze_image(image_path)
    
    if result:
        print("\n3. Displaying results...")
        display_analysis_result(result)
        
        print("\n4. Getting recent history...")
        history = get_analysis_history(3)
        
        return result
    else:
        print("❌ Analysis failed")
        return None

# Example usage - replace with your image path
# result = complete_workflow("path/to/your/car/image.jpg")

## 6. Web Interface Access

For a user-friendly interface, you can also access the web UI:

- **Streamlit UI**: http://localhost:8501
- **API Documentation**: http://localhost:8000/docs
- **MinIO Console**: http://localhost:9001 (admin/admin)

## Tips for Best Results

1. **Image Quality**: Use clear, well-lit images
2. **Angle**: Multiple angles provide better analysis
3. **Distance**: Ensure damage areas are clearly visible
4. **Resolution**: Higher resolution images improve detection accuracy

## Next Steps

- Explore the advanced notebook for batch processing
- Check the API documentation at /docs
- Try different images to understand the system's capabilities